Generate the random training set of inputs with their respective labels, for this the target vector $w_t$ is used to determine the label, in this way 3050 random inputs are generated quickly.

In [1]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import partial_trace, Statevector
from qiskit.circuit.library import ZGate, XGate
from qiskit import execute, Aer
from random import shuffle
import numpy as np
import random

In [2]:
def createSFblock(N, j):
    
    circuit = QuantumCircuit(N)
    
    state_decimal = format(j, '0'+str(N)+'b')[::-1] # inverse order due to Qiskit order: |q1q0>
    
    for qubit in range(N):
        if state_decimal[qubit] == '0':
            circuit.x(qubit)
    
    multiConrolledZ = ZGate().control(num_ctrl_qubits=N-1)
    circuit.append(multiConrolledZ, range(N))
    
    for qubit in range(N):
        if state_decimal[qubit] == '0':
            circuit.x(qubit)
    
    return circuit


In [3]:
def U_i(N, ki, circuit):
    
    # create equal superposition
    circuit.h(range(N))
    
    # apply SF blocks according to ki information
    vector_i = format(ki, '0'+str(2**N)+'b')
    for ij in range(len(vector_i)):
        if vector_i[ij] == '1': # n_j = 1, equivalent of having -1 in vector i
            circuit.compose(createSFblock(N, ij), inplace=True)  # the state i is the one that must have the phase of -1

def U_w(N, kw, circuit):
    
    # apply SF blocks according to kw information
    vector_w = format(kw, '0'+str(2**N)+'b')
    for wj in range(len(vector_w)):
        if vector_w[wj] == '1': # n_j = 1, equivalent of having -1 in vector w
            circuit.compose(createSFblock(N, wj), inplace=True)  # the state w is the one that must have the phase of -1
    
    # apply Hadamards
    circuit.h(range(N))
    
    # apply NOTs
    circuit.x(range(N))

In [4]:
def perceptron_brute_force(N, ki, kw, draw=False):
    
    # create circuit, N qubits + ancilla
    circuit = QuantumCircuit(N + 1, 1)
    
    # apply U_i
    U_i(N, ki, circuit)
    
    # apply U_w
    U_w(N, kw, circuit)
    
    # apply C^N X
    circuit.mcx(control_qubits=[i for i in range(N)], target_qubit=N)
    
    # measure the ancilla qubit
    circuit.measure(N, 0)
    
    # draw circuit
    if draw == True:
        display(circuit.draw('mpl'))
    
    return circuit

In [5]:
%%time

# Randomly generate input data

N = 4

kw_t = 64909                                  # cross-shaped pattern

data_positive = 50
data_negative = 3000
count_positive = 0
count_negative = 0
threshold = 0.5

chosen_ki = []

results = []                                  # vector of tuples

need_more = True
while need_more:
    ki = random.randint(0, 2**(2**N))
    vector_i = format(ki, '0'+str(2**N)+'b')
    
    if sum([int(i) for i in vector_i]) <= int(len(vector_i)/2): continue # not take the negative pattern (inverted images)
        
    if ki != kw_t and ki not in chosen_ki:           # not choose the target pattern and chose different inputs
        perceptron_circuit = perceptron_brute_force(N, ki, kw_t)
        perceptron_circuit.remove_final_measurements()
        full_state = Statevector(perceptron_circuit)
        partial_density_matrix_ancilla = partial_trace(full_state, range(N))
        probability = abs(partial_density_matrix_ancilla.data[1][1])
        if probability > threshold:
            if count_positive < data_positive:
                count_positive += 1
                results.append((ki, 1))
            else:
                continue
        else:
            if count_negative < data_negative:
                count_negative += 1
                results.append((ki, -1))
            else:
                continue
        
        chosen_ki.append(ki)
        
        if count_positive == data_positive and count_negative == data_negative:
            need_more = False

        print("\rcount_positive = {}, count_negative = {}".format(count_positive, count_negative), end="")
        
print()

count_positive = 50, count_negative = 3000
CPU times: total: 1h 20min 4s
Wall time: 1h 20min 20s


In [6]:
# shuffle the training set because most of the positive elements were left at the end
shuffle(results)

In [7]:
# save data in order to be able to reuse it
with open("training_set_3050.txt", "w") as file:
    file.write(str(results))

In [8]:
#with open("training_set_3050.txt", "r") as file:
#    results = eval(file.readline())